## Import important libraries
--------

In [ ]:
# Importing libraries 
import numpy as np
import random
import math
import random 
from collections import deque
import collections
import pickle

# for building DQN 
import tensorflow as tf
from keras import layers
from keras import Sequential
from keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam

# for plotting graphs
import matplotlib.pyplot as plt

# Clearing outputs
from IPython.display import clear_output

## Load the time matrix
#### Also, change some plotting parameters
---------

In [ ]:
# Loading the time matrix provided
Time_matrix = np.load("../input/rl-project-files/TM_lower.npy")

In [ ]:
# Change the plotting size
plt.rcParams["figure.figsize"] = (10,6)

## Defining the Environment

In [ ]:
# Defining hyperparameters
m = 5 # number of cities, ranges from 1 ..... m
t = 24 # number of hours, ranges from 0 .... t-1
d = 7  # number of days, ranges from 0 ... d-1
C = 5 # Per hour fuel and other costs
R = 9 # per hour revenue from a passenger

class CabDriver():
    
    '''Constructor function for the environment...'''
    def __init__(self):
        # Create the action space
        self.action_space = [[0,0]] + [[i,j] for i in range(1, m+1) for j in range(1, m+1) if i != j]
        
        # Create the state space 
        self.state_space = [[i,j,k] for i in range (1,m+1) for j in range(t) for k in range(d)]
        
        # Choose a random state
        self.state_init = self.state_space[random.randrange(0, len(self.state_space))]

        # Start the first round
        self.reset()


    ''' Function for converting the state into a one-hot encoded vector.
        
        @params: state: State to be one-hot encoded. It is a list [location, time, day]
        @returns: One-hot encoded form of given state
        
    '''
    def state_encod_arch2(self, state):
        location = state[0] # Store the current location
        time = state[1]     # Store the current time
        day = state[2]      # Store the current day
        
        # Create a vector for encoding the state
        state_encod = np.zeros((m+t+d, 1))
        
        # Encode the location information. Notice that location is from [1,2,3,4,5], so we need
        # to subtract 1 from the current location to one-hot encode it properly. 
        state_encod[location-1][0] = 1 
        
        # Encode the time information
        state_encod[m+time][0] = 1
        
        # Encode the day information
        state_encod[m+t+day][0] = 1
        
        # Return the state encoding
        return state_encod

    ''' Function for getting requests given a state. The number of requests
        is decided by the MDP formulation given in the problem statement. 
        
        @params: state: State (not in one-hot encoded form). It is a list [location, time, day]
        @returns: possible_actions_index: List of indices of possible actions.
                  These indices correspond to the index of that action in the action space
        @returns: actions: The actual actions (a list of lists, with each inner list consisting of two locations)
        
    '''
    def requests(self, state):
        # Determining the number of requests basis the location. 
        # Get the current location
        location = state[0]
        
        # Set number of requests to 0 for now...
        requests = 0
        
        # Get the number of requests as per MDP specified in problem statement
        if location == 1:
            requests = np.random.poisson(2)
        elif location == 2:
            requests = np.random.poisson(12)
        elif location == 3:
            requests = np.random.poisson(4)
        elif location == 4:
            requests = np.random.poisson(7)
        elif location == 5:
            requests = np.random.poisson(8)

        # Cap the total number of requests at 15
        if requests > 15:
            requests = 15

        # Randomly choose requests from the action space. Note that (0,0) is not a customer request. 
        possible_actions_index = random.sample(range(1, (m-1)*m +1), requests) 
        
        # Get the actions corresponding to those indices
        actions = [self.action_space[i] for i in possible_actions_index]
        
        # Finally append the lazy action - where the driver doesn't take any request at all
        possible_actions_index.append(0)
        actions.append([0,0])

        # Return the list of action indices, and the actions corresponding to those indices. 
        return possible_actions_index, actions   

    ''' This function gives out the reward for an action taken at a particular state
    
        @params: state: State (not in one-hot encoded form). It is a list [location, time, day]
        @params: action: The action taken from the action space 
        @params: Time_matrix: The time matrix that is used for deciding the time taken to go 
                 from one location to another, at a given time and a given day
                 
        @returns: A scalar, which is the reward the agent gets for taking the corresponding 
                  action from the given state. 
                  
    '''
    def reward_func(self, state, action, Time_matrix):
        # If the driver decides to not take any ride, he still incurs the fuel cost...
        if(action == [0,0]):
            return -1*C
        # However, if the driver decides to take a ride, we should calculate the reward he gets...
        else:
            # time_ip is the amount of time taken by driver to go from his current location to pickup location
            time_ip = int(Time_matrix[state[0]-1][action[0]-1][state[1]][state[2]])
            
            # Once the driver reaches the pickup location, we will have a new time and day. So we calculate that...
            new_time = state[1] + time_ip 
            new_day = state[2]
            new_day = (new_day + (new_time // 24)) % 7
            new_time = new_time % 24 
            
            # time_pq is the amount of tiem taken by the driver to go from his pickup location to destination location
            time_pq = int(Time_matrix[action[0]-1][action[1]-1][new_time][new_day])
            
            # Compute the reward as specified in the MDP
            reward = R * time_pq - C * (time_pq + time_ip)
            
            # Return the reward... 
            return reward
    
    ''' Function for determining the next state, given the current state.
        
        @params: state: State (not in one-hot encoded form). It is a list [location, time, day]
        @params: action: The action taken from the action space 
        @params: Time_matrix: The time matrix that is used for deciding the time taken to go 
                 from one location to another, at a given time and a given day
        @returns: The next state and the time elapsed for taking this action.
        
    '''
    def next_state_func(self, state, action, Time_matrix):
        # If the driver decided to not entertain any ride, his location is same
        # Only the time (and possibly the day) gets increased by one hour
        if(action == [0,0]):
            # New location is same as old location
            new_location = state[0]   
            
            # Compute the new time 
            new_time = state[1] + 1
            
            # Compute the new day
            new_day = state[2]         
            new_day = new_day + (new_time//24)
            
            # Take modulos of new_day and new_time to ensure they are within limits... 
            new_day = new_day % 7
            new_time = new_time % 24
            
            # Create the new state
            new_state = [new_location ,new_time, new_day]
            
            # Return the new state and action
            return [new_state, 1] # 1 hour is the time elapsed
        else:
            # As usual, time_ip is the time taken by driver to go from current location to pickup location
            time_ip = int(Time_matrix[state[0]-1][action[0]-1][state[1]][state[2]])
            
            # Compute the new time and day, since we will use updated state to compute the time taken for 
            # going from pickup location to destination location... 
            new_time = state[1] + time_ip
            new_day = state[2]
            new_day = (new_day + (new_time // 24)) % 7
            new_time = new_time % 24 
            
            # Again, time_pq is the time taken to go from pickup location to destination location
            time_pq = int(Time_matrix[action[0]-1][action[1]-1][new_time][new_day])
            
            # Compute the total time elapsed...
            total_time = time_ip + time_pq 
            
            # Compute the new state
            new_location = action[1]
            new_time = state[1] + total_time
            new_day = state[2]
            new_day = ((new_day + (new_time//24))%7)
            new_time = new_time % 24 
            next_state = [new_location, new_time, new_day]
            
            # Return the new state and total time elapsed...
            return [next_state, total_time] 

    ''' Function for resetting the environment. 
        
        @returns: The action space, state space and the initial space (not one-hot encoded) 
        
    '''
    def reset(self):
        return self.action_space, self.state_space, self.state_init

## Creating the agent
----------------------

In [ ]:
class DQNAgent:
    ''' Constructor for the DQN Agent. 
        
        @params: state_size: The size of the one-hot encoded state-vector
        @params: action_size: The size of the action space
    '''
    def __init__(self, state_size, action_size):
        self.state_size = state_size     # Specify the state size
        self.action_size = action_size   # Specify the action size

        self.discount_factor = 0.95      # The discount factor (gamma)
        self.learning_rate = 0.01        # The learning rate for training the NN
        self.epsilon = 0.999             # Initial epsilon, high epsilon means high exploration  
        self.epsilon_decay = 0.995       # The epsilon decay factor. epsilon is multiplied by epsilon_decay every 4 episodes 
        self.epsilon_min = 0.01          # Minimum epsilon. The epsilon cannot go below this   
        
        self.batch_size = 128            # The batch size for training
        
        self.memory = deque(maxlen=2000) # This structure is used as the memory replay, storing [s,a,r,s',done]

        # Create the model to be used by the agent. It would be a neural network
        self.model = self.build_model()

    ''' This function builds the Neural Network using which will be deciding the 
        the actions of the agent. 
        
        @returns: model: The Keras Neural Network model
    '''
    def build_model(self):
        # We create a sequential model 
        model = Sequential()
        
        # Create the layers. There are 3 layers, with the first two layers having
        # ReLU activation function and the last one having a linear activation function
        model.add(Dense(64, input_dim = self.state_size, activation = tf.keras.layers.ReLU()))
        model.add(Dense(32, activation = tf.keras.layers.ReLU()))
        model.add(Dense(self.action_size, activation = 'linear'))
        
        # Compile the model. We will be using Adam as the optimizer, with MSE as the loss
        model.compile(loss='mse', optimizer=Adam(learning_rate = self.learning_rate))
        
        # Return the model
        return model

    ''' Function for converting the state into a one-hot encoded vector.
 
         This function is identical to the state encoder of the environment, except the fact
         that the one-hot encoded vector of the state being returned is a horizontal vector
         instead of a vertical vector. 
     
         @params: state: State to be one-hot encoded. It is a list [location, time, day]
         @returns: One-hot encoded form of given state
 
    '''
    def state_encoder(self, state):
        location = state[0]       # Store the current location
        time = state[1]           # Store the current time
        day = state[2]            # Store the current day
        
        # Create the one hot encoded vector
        state_encod = np.zeros((m+t+d, 1)) 
        
        state_encod[location-1][0] = 1   # One-hot encode the location     
        state_encod[m+time][0] = 1       # one-hot encode the time
        state_encod[m+t+day][0] = 1      # One-hot encode the day
        
        # Make the vector a horizontal vector
        state_encod = state_encod.reshape((1, state_encod.shape[0]))
        
        # Return the one-hot encoded vector
        return state_encod

    ''' This function is used for getting the action from the model. This is, in some
        sense, the policy. To be more precise, it is an epsilon-greedy policy. The 
        epsilon can be controlled using the parameters. 
    
        @params: state: State in which te driver is currently at.
        @params: actions_ind: The list of possible action indices.
        @returns: the action index that is being taken by the policy... 
    '''
    def get_action(self, state, actions_ind):
        # Check if the action we need to take is a randomized one... 
        if np.random.rand() <= self.epsilon:
            return random.choice(actions_ind)
        
        # If not, we take the best action as per the model
        else:
            # Encode the state
            encoded_state = self.state_encoder(state)    
            
            # Get the Q value of all actions for the states
            q_value = self.model.predict(encoded_state)  
            
            # Filter out the Q-values to include only action which can be taken
            required_q_values = [q_value[0][j] for j in actions_ind]   

            # Return the index of the maximum action (having maximum Q-value) that the model predicts 
            return actions_ind[np.argmax(required_q_values)]  

    ''' This function appends a sample into the memory replay buffer.
    
        Note that we don't decay the epsilon every time a sample has been appended, we will
        rather decay them once the episode ends. Otherwise, our epsilon would decay very 
        fast. 
        
        @params: state: The state (not one-hot encoded) to be pushed to the buffer
        @params: action: The index of the action (in sync with that in action_space) being taken
        @params: reward: The reward obtained for taking the given action from the given state
        @params: next_state: The next state to which the agent goes after taking action from state 
        @params: done: A boolean. True means the episode has ended, otherwise False. 
        
    '''
    def append_sample(self, state, action, reward, next_state, done):
        # Append <s,a,r,s',done> to the mmeory replay buffer
        self.memory.append((state, action, reward, next_state, done))
    
    ''' This function trains the model using samples from the memory replay buffer
        It doesn't take in any parameters and doesn't return anything
        
    '''
    def train_model(self):
        # Perform training only if enough samples are available
        if len(self.memory) > self.batch_size:
            
            # Sample batch from the memory
            mini_batch = random.sample(self.memory, self.batch_size)
            
            # Create arrays holding the states s and s' in a sample <s,a,r,s',done>
            update_output = np.zeros((self.batch_size, self.state_size))
            update_input = np.zeros((self.batch_size, self.state_size))
            
            # Arrays for storing the actions, rewards, and dones for different samples
            action, reward, done = [], [], []
            
            # Now, we start processing the samples from the batch
            for i in range(self.batch_size):
                # Get the <s,a,r,s',done> for the i'th entry from the mini batch
                c_state, c_action, c_reward, c_next_state, c_done = mini_batch[i]
                
                # Append the action, reward, done for the current sample
                action.append(c_action)
                reward.append(c_reward)
                done.append(c_done)
                
                # Create state encoding for initial state
                update_input[i] = self.state_encoder(c_state)
                
                # Create encoding for the final state
                update_output[i] = self.state_encoder(c_next_state)
                
            # Predict the Q-values for given states (s) of the mini-batch
            target = self.model.predict(update_input)
            
            # Predict the Q-values for the given next states (s') of the mini-batch
            Q_output_states = self.model.predict(update_output)
            
            # Now, update the targets for the actions taken from state s
            for i in range(self.batch_size):
                if(done[i]):
                    target[i][action[i]] = reward[i]
                else:
                    target[i][action[i]] = reward[i] + self.discount_factor * np.max(Q_output_states[i])              
                
            # Fit the model and track the loss values
            self.model.fit(update_input, target, batch_size = self.batch_size, epochs = 1, verbose = 0)

### DQN block

In [ ]:
# Define total number of episodes  
total_episodes = 1900

# Create a temporary environment. This environment is used for getting the action space and state space
env_tmp = CabDriver()

# Getting the action and state space from the temporary environment
action_space, state_space, initial_state = env_tmp.reset()

# Define state size
state_sz = m + t + d

# Define the size of action space
action_sz = len(action_space)

# Call the DQN Agent
agent = DQNAgent(state_sz, action_sz)

# Create a list for tracking rewards. We use it for plotting purposes. 
reward_tracker = []

# Print the agent model summary
print(agent.model.summary())

In [ ]:
# Mention the state whose Q-values would be tracked
state_to_track = [1,0,0]
track_state = agent.state_encoder(state_to_track)

# Mention the action index whose Q-values are to be tracked. So, we track Q(state_to_track, action_space[track_action_dx])
track_action_idx = 1 # This is the action [1, 2]

# Create a list for tracking Q-values. We use it for plotting purposes. 
qval_tracker = [] 

''' Function for getting the Q-value for given state, action pair.
    
    @params: track_state: State (not one-hot encoded) whose Q-value is needed
    @params: track_action_idx: The action index (as in action_space) being taken. 
    @returns: Returns Q-value Q(track_state, action_space[track_action_dx])
'''
def get_qval(track_state, track_action_idx):
    # Predict the Q-values of all actions possible from track_state
    res_qval = agent.model.predict(track_state)
    
    # Get the Q-value for the action required
    qval = res_qval[0][track_action_idx]
    
    # Return the corresponding Q-value
    return qval

In [ ]:
for episode in range(total_episodes):
    # Print the current episode
    print("In episode " + str(episode))
    
    # Call the environment
    env = CabDriver()
    
    # Call all the initialised variables of the environment
    action_space, state_space, current_state = env.reset()
    
    # Variable for keeping track of whether the episode has ended or not
    done = False
    
    total_time = 0  # Total time elapsed for the current episode
    sum_rew = 0   # Total reward obtained in the current episode 
    
    # Now, run the episode
    while done == False:
        # Get the actions possible from the current state
        actions_possible_ind, possible_action = env.requests(current_state)
        
        # Get the action index that the agent would like to take
        agent_action_index = agent.get_action(current_state, actions_possible_ind)
        
        # Get the actual action from the agent_action_index and the action_space
        agent_action = action_space[agent_action_index]
        
        # Get the reward for taking this action from the current state
        reward = env.reward_func(current_state, agent_action, Time_matrix)
        
        # Update total reward obtained
        sum_rew += reward
        
        # Get the new state and the total time elapsed
        new_state, time_elapsed = env.next_state_func(current_state, agent_action, Time_matrix)
        
        # Update the total time elapsed
        total_time += time_elapsed
        
        # If the agent took action that exceeded maximum time, ignore that action
        if(total_time >= 24*30):
            done = True
            
        # Otherwise, append the sample, train the model and set the current state as the new state
        else:
            agent.append_sample(current_state, agent_action_index, reward, new_state, done)
            agent.train_model()
            current_state = new_state
    
    # Every 4 episodes, we do the decay of epsilon.
    # By doing so, at around 1900 episodes, the epsilon would 0.1
    if(((episode + 1) % 4) == 0):
        if(agent.epsilon > agent.epsilon_min):
            agent.epsilon *= agent.epsilon_decay
         
    # To keep notebook clean, we just clear out any plots that we 
    # have plotted after every 100 episodes 
    if(((episode + 1) % 100)  == 0):
        clear_output(wait = True)
        
    # Append the reward to reward_tracker
    reward_tracker.append(sum_rew)
    
    # Append the Q-value to the Q-value tracker.
    qval_tracker.append(get_qval(track_state, track_action_idx))
    
    # Plot the rewards and Q-values after every 20 episodes.
    if(((episode+1) % 20) == 0):
        # Plot the rewards
        plt.plot(reward_tracker)
        title = "Current Epsilon: " + str(agent.epsilon)
        plt.title(title)
        plt.show()
        
        # Plot the Q-values
        plt.plot(qval_tracker)
        title = "QValue Plot for state " + str(state_to_track) + " for action " + str(action_space[track_action_idx])
        plt.title(title)
        plt.show()

In [50]:
# Save the converged model as a h5 file. 
# I didn't use pickle because it failed (pickle works for sequences only...)
agent.model.save('Converged_1900.h5')

In [ ]:
# Function for plotting smoothened form of the reward function
# This code was taken from: https://stackoverflow.com/questions/5283649/plot-smooth-line-with-pyplot

def smooth(scalars, weight):  # Weight between 0 and 1
    last = scalars[0]  # First value in the plot (first timestep)
    smoothed = list()
    for point in scalars:
        smoothed_val = last * weight + (1 - weight) * point  # Calculate smoothed value
        smoothed.append(smoothed_val)                        # Save it
        last = smoothed_val                                  # Anchor the last smoothed value
        
    return smoothed


plt.plot(smooth(reward_tracker, 0.95))
plt.title("Smooth plot of rewards obtained")
plt.show()

## Plot of epsilon-decay function being used
------------

<div class="alert alert-block alert-info">
Try building a similar epsilon-decay function for your model.
</div>

In [ ]:
# List to keep track of values
track_values = [1]

# Loop for simulating the epsilon-decay
for i in range(total_episodes):
    last_val = track_values[-1]
    if((i+1) % 4 == 0):
        new_val = last_val * agent.epsilon_decay
        track_values.append(new_val)
    else:
        track_values.append(last_val)
        
# Plot the decay portion. Note that the decay is applied
# after every 4 episodes...
plt.plot(track_values)
plt.title("Epsilon decay plot")
plt.show()

---------